# Libraries

In [1]:
import selenium 
import csv
import re
import pandas as pd
import ast
import numpy as np
import math

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options

import time

# Parser
- Convert original non-queryable names to queryable.

In [2]:
data = pd.read_csv('task_2_with_pub.csv')
data 

,cas,url,common,preferred,brand,inn,pubchem,who_atc,originator,stereochemistry,...,us_market_year,us_prev_market,first_year_approval,fda_links,target_list,condition_list,patent_list,pubmed,pmid,mesh
0,585-86-4,https://drugs.ncats.io/drug/L2B0WJF7ZY,"['LACTITOL ANHYDROUS', 'LACTOBIOSIT', 'MIRUHEN...",LACTITOL,['IMPORTAL'],6414.0,157355.0,A06AD12,"Senderens, J.B.",ABSOLUTE,...,['2020'],0,2020.0,['NDA211281'],[],"['Constipation', 'Hepatic Encephalopathy']","['JP2000201631A', 'JP2001511442A', 'JP20041617...",['https://www.ncbi.nlm.nih.gov/pubmed/19087388...,"['19087388', '16481971', '16498257', '16553741...",C014635
1,1289023-67-1,https://drugs.ncats.io/drug/997WVV895X,"['BHV-3000', 'RIMEGEPANT [INN]', 'RIMEGEPANT [...",RIMEGEPANT,[],9751.0,51049968.0,NaN,Bristol-Myers Squibb,ABSOLUTE,...,['2020'],0,2020.0,['NDA212728'],['Calcitonin gene-related peptide type 1 recep...,['Migraine Disorders'],"['20110251223', '20120245356']",['https://www.ncbi.nlm.nih.gov/pubmed/26650258'],['26650258'],NaN
2,738606-46-7,https://drugs.ncats.io/drug/1EJ6Z6Q368,"['PENTADECANEDIOIC ACID, 8-HYDROXY-2,2,14,14-T...",BEMPEDOIC ACID,[],9891.0,10472693.0,NaN,Esperion Therapeutics,ACHIRAL,...,['2020'],0,2020.0,['NDA211617'],"['ATP-citrate synthase', 'AMPK alpha1$$alpha2']","['Hypercholesterolemia', 'Dyslipidemias']","['20050043278', '20070179120', '7335799', '781...",['https://www.ncbi.nlm.nih.gov/pubmed/1371749'],['1371749'],NaN
3,71675-85-9,https://drugs.ncats.io/drug/8110R61I4U,"['AMISULPRIDE [EP]', 'AMISULPRIDE [WHO-DD]', '...",AMISULPRIDE,"['SULAMID', 'SOLIAN', 'DENIBAN', 'SOCIAN']",4960.0,2159.0,N05AL05,Sanofi,RACEMIC,...,['2020'],0,2020.0,['NDA209510'],"['Dopamine D3 receptor', 'Serotonin 7 (5-HT7) ...","['Schizophrenia', 'Psychotic symptoms']","['JP2001501192A', 'JP2002527464A', 'JP4178032B2']",['https://www.ncbi.nlm.nih.gov/pubmed/1354163'...,"['1354163', '11803729', '12693427', '12442883'...",NaN
4,308242-62-8,https://drugs.ncats.io/drug/7V4A8U16MB,"['REMIMAZOLAM [INN]', 'REMIMAZOLAM [WHO-DD]']",REMIMAZOLAM,[],9232.0,9867812.0,NaN,Glaxo Smith Kline,ABSOLUTE,...,['2020'],0,2020.0,['NDA212295'],['GABA-A receptor; benzodiazepine site'],[],[],[],[],C522201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,9005-35-0,https://drugs.ncats.io/drug/8P20S56HZI,"['ALGINATES: AMMONIUM, CALCIUM, POTASSIUM AND ...",CALCIUM ALGINATE,[],NaN,44630049.0,B02BC08,"Moissan, H.",NaN,...,NaN,0,1921.0,[],[],"['Cystic Fibrosis', 'Hypocalcemia', 'Chronic O...","['20090030026', '20130174651', '3966773', '397...",['https://www.ncbi.nlm.nih.gov/pubmed/1220513'...,"['1220513', '11572466', '11513607', '11592964'...",NaN
3645,9006-68-2,https://drugs.ncats.io/drug/8B8GHO5O27,"['POLYBENZARSOL [INN]', 'POLYBENZARSOL [MI]']",POLYBENZARSOL,['BENZODOL'],727.0,68716.0,NaN,"Butlerov, A.",NaN,...,NaN,0,1921.0,[],['DNA'],['Warts'],"['3944600', '3954756', '3969409', '3976653', '...",['https://www.ncbi.nlm.nih.gov/pubmed/1597190'...,"['1597190', '14666255', '14656365', '14979077'...",NaN
3646,9011-04-5,https://drugs.ncats.io/drug/4C905MSK4W,"['HEXADIMETHRINE BROMIDE [INN]', ""POLY(N,N,N',...",HEXADIMETHRINE BROMIDE,['POLYBRENE'],719.0,24769.0,NaN,NaN,NaN,...,NaN,1,1959.0,[],[],[],[],[],[],NaN
3647,8006-90-4,https://drugs.ncats.io/drug/AV092KU4JH,"['PEPPERMINT OIL', 'PEPPERMINT OIL YAKIMA', 'P...",PEPPERMINT OIL,[],NaN,6850741.0,NaN,NaN,NaN,...,NaN,0,1921.0,[],[],[],[],[],[],C015424


In [3]:
data['pubchem'] = data['pubchem'].astype('int64')
pubchem_cid_arr = data['pubchem'].to_list()

def get_pubchem_url(cid):
    return f'https://pubchem.ncbi.nlm.nih.gov/compound/{cid}'

links_arr = list(map(get_pubchem_url, pubchem_cid_arr))

# PubChem CAS Scraper

In [4]:
def setup_webdriver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def get_cas_numbers(link, driver):
    cas_numbers = {"CAS": "N/A", "Deprecated CAS": "N/A"}  # Default in case of failure

    try:
        driver.get(link)
        # Wait and locate the CAS number section
        try:
            cas_elements = WebDriverWait(driver, 60).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'section#CAS div.break-words')))
            cas_numbers["CAS"] = ', '.join([el.text.strip() for el in cas_elements])
        except (NoSuchElementException, TimeoutException):
            cas_numbers["CAS"] = "N/A"  # CAS number not found

        # Wait and locate the Deprecated CAS number section, if present
        try:
            deprecated_cas_elements = driver.find_elements(By.CSS_SELECTOR, 'section#Deprecated-CAS div.break-words')
            # deprecated_cas_elements = WebDriverWait(driver, 3).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'section#Deprecated-CAS div.break-words')))
            cas_numbers["Deprecated CAS"] = ', '.join([el.text.strip() for el in deprecated_cas_elements])
            
        except (NoSuchElementException, TimeoutException):
            cas_numbers["Deprecated CAS"] = "N/A"  # Deprecated CAS number not found
            
    except Exception as e:
        print(f"Error retrieving CAS numbers for {link}: {e}")

    return cas_numbers

def get_cas_numbers_concurrently(links):
    cas_matches = []
    driver = setup_webdriver()  # Initialize the WebDriver once

    try:
        for link in links:
            cas_matches.append(get_cas_numbers(link, driver))

    finally:
        driver.quit()  # Make sure to quit the WebDriver

    return cas_matches      

In [5]:
cas_numbers = get_cas_numbers_concurrently(links_arr)

In [6]:
df_cas = pd.DataFrame(cas_numbers, columns=['CAS', 'Deprecated CAS'])
df_cas

,CAS,Deprecated CAS
0,585-86-4,
1,"1289023-67-1, 1374024-48-2",
2,738606-46-7,
3,"71675-85-9, 53583-79-2",
4,308242-62-8,
...,...,...
3644,9005-35-0,"37228-92-5, 9019-42-5, 9019-43-6, 9060-20-2"
3645,N/A,
3646,N/A,
3647,8006-90-4,


In [7]:
final_df = pd.concat([data, df_cas], ignore_index=True, sort=False, axis=1)
final_df

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,585-86-4,https://drugs.ncats.io/drug/L2B0WJF7ZY,"['LACTITOL ANHYDROUS', 'LACTOBIOSIT', 'MIRUHEN...",LACTITOL,['IMPORTAL'],6414.0,157355,A06AD12,"Senderens, J.B.",ABSOLUTE,...,2020.0,['NDA211281'],[],"['Constipation', 'Hepatic Encephalopathy']","['JP2000201631A', 'JP2001511442A', 'JP20041617...",['https://www.ncbi.nlm.nih.gov/pubmed/19087388...,"['19087388', '16481971', '16498257', '16553741...",C014635,585-86-4,
1,1289023-67-1,https://drugs.ncats.io/drug/997WVV895X,"['BHV-3000', 'RIMEGEPANT [INN]', 'RIMEGEPANT [...",RIMEGEPANT,[],9751.0,51049968,NaN,Bristol-Myers Squibb,ABSOLUTE,...,2020.0,['NDA212728'],['Calcitonin gene-related peptide type 1 recep...,['Migraine Disorders'],"['20110251223', '20120245356']",['https://www.ncbi.nlm.nih.gov/pubmed/26650258'],['26650258'],NaN,"1289023-67-1, 1374024-48-2",
2,738606-46-7,https://drugs.ncats.io/drug/1EJ6Z6Q368,"['PENTADECANEDIOIC ACID, 8-HYDROXY-2,2,14,14-T...",BEMPEDOIC ACID,[],9891.0,10472693,NaN,Esperion Therapeutics,ACHIRAL,...,2020.0,['NDA211617'],"['ATP-citrate synthase', 'AMPK alpha1$$alpha2']","['Hypercholesterolemia', 'Dyslipidemias']","['20050043278', '20070179120', '7335799', '781...",['https://www.ncbi.nlm.nih.gov/pubmed/1371749'],['1371749'],NaN,738606-46-7,
3,71675-85-9,https://drugs.ncats.io/drug/8110R61I4U,"['AMISULPRIDE [EP]', 'AMISULPRIDE [WHO-DD]', '...",AMISULPRIDE,"['SULAMID', 'SOLIAN', 'DENIBAN', 'SOCIAN']",4960.0,2159,N05AL05,Sanofi,RACEMIC,...,2020.0,['NDA209510'],"['Dopamine D3 receptor', 'Serotonin 7 (5-HT7) ...","['Schizophrenia', 'Psychotic symptoms']","['JP2001501192A', 'JP2002527464A', 'JP4178032B2']",['https://www.ncbi.nlm.nih.gov/pubmed/1354163'...,"['1354163', '11803729', '12693427', '12442883'...",NaN,"71675-85-9, 53583-79-2",
4,308242-62-8,https://drugs.ncats.io/drug/7V4A8U16MB,"['REMIMAZOLAM [INN]', 'REMIMAZOLAM [WHO-DD]']",REMIMAZOLAM,[],9232.0,9867812,NaN,Glaxo Smith Kline,ABSOLUTE,...,2020.0,['NDA212295'],['GABA-A receptor; benzodiazepine site'],[],[],[],[],C522201,308242-62-8,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,9005-35-0,https://drugs.ncats.io/drug/8P20S56HZI,"['ALGINATES: AMMONIUM, CALCIUM, POTASSIUM AND ...",CALCIUM ALGINATE,[],NaN,44630049,B02BC08,"Moissan, H.",NaN,...,1921.0,[],[],"['Cystic Fibrosis', 'Hypocalcemia', 'Chronic O...","['20090030026', '20130174651', '3966773', '397...",['https://www.ncbi.nlm.nih.gov/pubmed/1220513'...,"['1220513', '11572466', '11513607', '11592964'...",NaN,9005-35-0,"37228-92-5, 9019-42-5, 9019-43-6, 9060-20-2"
3645,9006-68-2,https://drugs.ncats.io/drug/8B8GHO5O27,"['POLYBENZARSOL [INN]', 'POLYBENZARSOL [MI]']",POLYBENZARSOL,['BENZODOL'],727.0,68716,NaN,"Butlerov, A.",NaN,...,1921.0,[],['DNA'],['Warts'],"['3944600', '3954756', '3969409', '3976653', '...",['https://www.ncbi.nlm.nih.gov/pubmed/1597190'...,"['1597190', '14666255', '14656365', '14979077'...",NaN,N/A,
3646,9011-04-5,https://drugs.ncats.io/drug/4C905MSK4W,"['HEXADIMETHRINE BROMIDE [INN]', ""POLY(N,N,N',...",HEXADIMETHRINE BROMIDE,['POLYBRENE'],719.0,24769,NaN,NaN,NaN,...,1959.0,[],[],[],[],[],[],NaN,N/A,
3647,8006-90-4,https://drugs.ncats.io/drug/AV092KU4JH,"['PEPPERMINT OIL', 'PEPPERMINT OIL YAKIMA', 'P...",PEPPERMINT OIL,[],NaN,6850741,NaN,NaN,NaN,...,1921.0,[],[],[],[],[],[],C015424,8006-90-4,


In [8]:
final_df.columns=['url', 'common', 'preferred', 'brand', 'inn', 'pubchem', 'who_atc', 'cas', 'originator', 'stereochemistry', 'formula', 'mw', 'investigational', 'us_market', 'us_market_year', 'us_prev_market', 'first_year_approval', 'fda_links', 'target_list', 'condition_list', 'patent_list', 'pubmed', 'pmid', 'mesh', 'CAS', 'Deprecated CAS']
# final_df.to_csv('margaret_missing_cas._identified.csv', index=False)

In [9]:
# final_df['Original Molecule'] = final_df['Parsed Molecule']
# final_df = final_df[['Original Molecule', 'Parsed Molecule', 'Link', 'Result Type', 'CAS', 'Deprecated CAS', 'Compound/Substance']]
final_df

,url,common,preferred,brand,inn,pubchem,who_atc,cas,originator,stereochemistry,...,first_year_approval,fda_links,target_list,condition_list,patent_list,pubmed,pmid,mesh,CAS,Deprecated CAS
0,585-86-4,https://drugs.ncats.io/drug/L2B0WJF7ZY,"['LACTITOL ANHYDROUS', 'LACTOBIOSIT', 'MIRUHEN...",LACTITOL,['IMPORTAL'],6414.0,157355,A06AD12,"Senderens, J.B.",ABSOLUTE,...,2020.0,['NDA211281'],[],"['Constipation', 'Hepatic Encephalopathy']","['JP2000201631A', 'JP2001511442A', 'JP20041617...",['https://www.ncbi.nlm.nih.gov/pubmed/19087388...,"['19087388', '16481971', '16498257', '16553741...",C014635,585-86-4,
1,1289023-67-1,https://drugs.ncats.io/drug/997WVV895X,"['BHV-3000', 'RIMEGEPANT [INN]', 'RIMEGEPANT [...",RIMEGEPANT,[],9751.0,51049968,NaN,Bristol-Myers Squibb,ABSOLUTE,...,2020.0,['NDA212728'],['Calcitonin gene-related peptide type 1 recep...,['Migraine Disorders'],"['20110251223', '20120245356']",['https://www.ncbi.nlm.nih.gov/pubmed/26650258'],['26650258'],NaN,"1289023-67-1, 1374024-48-2",
2,738606-46-7,https://drugs.ncats.io/drug/1EJ6Z6Q368,"['PENTADECANEDIOIC ACID, 8-HYDROXY-2,2,14,14-T...",BEMPEDOIC ACID,[],9891.0,10472693,NaN,Esperion Therapeutics,ACHIRAL,...,2020.0,['NDA211617'],"['ATP-citrate synthase', 'AMPK alpha1$$alpha2']","['Hypercholesterolemia', 'Dyslipidemias']","['20050043278', '20070179120', '7335799', '781...",['https://www.ncbi.nlm.nih.gov/pubmed/1371749'],['1371749'],NaN,738606-46-7,
3,71675-85-9,https://drugs.ncats.io/drug/8110R61I4U,"['AMISULPRIDE [EP]', 'AMISULPRIDE [WHO-DD]', '...",AMISULPRIDE,"['SULAMID', 'SOLIAN', 'DENIBAN', 'SOCIAN']",4960.0,2159,N05AL05,Sanofi,RACEMIC,...,2020.0,['NDA209510'],"['Dopamine D3 receptor', 'Serotonin 7 (5-HT7) ...","['Schizophrenia', 'Psychotic symptoms']","['JP2001501192A', 'JP2002527464A', 'JP4178032B2']",['https://www.ncbi.nlm.nih.gov/pubmed/1354163'...,"['1354163', '11803729', '12693427', '12442883'...",NaN,"71675-85-9, 53583-79-2",
4,308242-62-8,https://drugs.ncats.io/drug/7V4A8U16MB,"['REMIMAZOLAM [INN]', 'REMIMAZOLAM [WHO-DD]']",REMIMAZOLAM,[],9232.0,9867812,NaN,Glaxo Smith Kline,ABSOLUTE,...,2020.0,['NDA212295'],['GABA-A receptor; benzodiazepine site'],[],[],[],[],C522201,308242-62-8,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,9005-35-0,https://drugs.ncats.io/drug/8P20S56HZI,"['ALGINATES: AMMONIUM, CALCIUM, POTASSIUM AND ...",CALCIUM ALGINATE,[],NaN,44630049,B02BC08,"Moissan, H.",NaN,...,1921.0,[],[],"['Cystic Fibrosis', 'Hypocalcemia', 'Chronic O...","['20090030026', '20130174651', '3966773', '397...",['https://www.ncbi.nlm.nih.gov/pubmed/1220513'...,"['1220513', '11572466', '11513607', '11592964'...",NaN,9005-35-0,"37228-92-5, 9019-42-5, 9019-43-6, 9060-20-2"
3645,9006-68-2,https://drugs.ncats.io/drug/8B8GHO5O27,"['POLYBENZARSOL [INN]', 'POLYBENZARSOL [MI]']",POLYBENZARSOL,['BENZODOL'],727.0,68716,NaN,"Butlerov, A.",NaN,...,1921.0,[],['DNA'],['Warts'],"['3944600', '3954756', '3969409', '3976653', '...",['https://www.ncbi.nlm.nih.gov/pubmed/1597190'...,"['1597190', '14666255', '14656365', '14979077'...",NaN,N/A,
3646,9011-04-5,https://drugs.ncats.io/drug/4C905MSK4W,"['HEXADIMETHRINE BROMIDE [INN]', ""POLY(N,N,N',...",HEXADIMETHRINE BROMIDE,['POLYBRENE'],719.0,24769,NaN,NaN,NaN,...,1959.0,[],[],[],[],[],[],NaN,N/A,
3647,8006-90-4,https://drugs.ncats.io/drug/AV092KU4JH,"['PEPPERMINT OIL', 'PEPPERMINT OIL YAKIMA', 'P...",PEPPERMINT OIL,[],NaN,6850741,NaN,NaN,NaN,...,1921.0,[],[],[],[],[],[],C015424,8006-90-4,


# Export Dataframe as CSV

In [10]:
# final_df_csv = final_df.to_csv('pubchem_longer_wait.csv', index = False) 
final_df.to_csv('task_2_pub_identified_60s.csv', index=False)

In [11]:
final_df

,url,common,preferred,brand,inn,pubchem,who_atc,cas,originator,stereochemistry,...,first_year_approval,fda_links,target_list,condition_list,patent_list,pubmed,pmid,mesh,CAS,Deprecated CAS
0,585-86-4,https://drugs.ncats.io/drug/L2B0WJF7ZY,"['LACTITOL ANHYDROUS', 'LACTOBIOSIT', 'MIRUHEN...",LACTITOL,['IMPORTAL'],6414.0,157355,A06AD12,"Senderens, J.B.",ABSOLUTE,...,2020.0,['NDA211281'],[],"['Constipation', 'Hepatic Encephalopathy']","['JP2000201631A', 'JP2001511442A', 'JP20041617...",['https://www.ncbi.nlm.nih.gov/pubmed/19087388...,"['19087388', '16481971', '16498257', '16553741...",C014635,585-86-4,
1,1289023-67-1,https://drugs.ncats.io/drug/997WVV895X,"['BHV-3000', 'RIMEGEPANT [INN]', 'RIMEGEPANT [...",RIMEGEPANT,[],9751.0,51049968,NaN,Bristol-Myers Squibb,ABSOLUTE,...,2020.0,['NDA212728'],['Calcitonin gene-related peptide type 1 recep...,['Migraine Disorders'],"['20110251223', '20120245356']",['https://www.ncbi.nlm.nih.gov/pubmed/26650258'],['26650258'],NaN,"1289023-67-1, 1374024-48-2",
2,738606-46-7,https://drugs.ncats.io/drug/1EJ6Z6Q368,"['PENTADECANEDIOIC ACID, 8-HYDROXY-2,2,14,14-T...",BEMPEDOIC ACID,[],9891.0,10472693,NaN,Esperion Therapeutics,ACHIRAL,...,2020.0,['NDA211617'],"['ATP-citrate synthase', 'AMPK alpha1$$alpha2']","['Hypercholesterolemia', 'Dyslipidemias']","['20050043278', '20070179120', '7335799', '781...",['https://www.ncbi.nlm.nih.gov/pubmed/1371749'],['1371749'],NaN,738606-46-7,
3,71675-85-9,https://drugs.ncats.io/drug/8110R61I4U,"['AMISULPRIDE [EP]', 'AMISULPRIDE [WHO-DD]', '...",AMISULPRIDE,"['SULAMID', 'SOLIAN', 'DENIBAN', 'SOCIAN']",4960.0,2159,N05AL05,Sanofi,RACEMIC,...,2020.0,['NDA209510'],"['Dopamine D3 receptor', 'Serotonin 7 (5-HT7) ...","['Schizophrenia', 'Psychotic symptoms']","['JP2001501192A', 'JP2002527464A', 'JP4178032B2']",['https://www.ncbi.nlm.nih.gov/pubmed/1354163'...,"['1354163', '11803729', '12693427', '12442883'...",NaN,"71675-85-9, 53583-79-2",
4,308242-62-8,https://drugs.ncats.io/drug/7V4A8U16MB,"['REMIMAZOLAM [INN]', 'REMIMAZOLAM [WHO-DD]']",REMIMAZOLAM,[],9232.0,9867812,NaN,Glaxo Smith Kline,ABSOLUTE,...,2020.0,['NDA212295'],['GABA-A receptor; benzodiazepine site'],[],[],[],[],C522201,308242-62-8,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,9005-35-0,https://drugs.ncats.io/drug/8P20S56HZI,"['ALGINATES: AMMONIUM, CALCIUM, POTASSIUM AND ...",CALCIUM ALGINATE,[],NaN,44630049,B02BC08,"Moissan, H.",NaN,...,1921.0,[],[],"['Cystic Fibrosis', 'Hypocalcemia', 'Chronic O...","['20090030026', '20130174651', '3966773', '397...",['https://www.ncbi.nlm.nih.gov/pubmed/1220513'...,"['1220513', '11572466', '11513607', '11592964'...",NaN,9005-35-0,"37228-92-5, 9019-42-5, 9019-43-6, 9060-20-2"
3645,9006-68-2,https://drugs.ncats.io/drug/8B8GHO5O27,"['POLYBENZARSOL [INN]', 'POLYBENZARSOL [MI]']",POLYBENZARSOL,['BENZODOL'],727.0,68716,NaN,"Butlerov, A.",NaN,...,1921.0,[],['DNA'],['Warts'],"['3944600', '3954756', '3969409', '3976653', '...",['https://www.ncbi.nlm.nih.gov/pubmed/1597190'...,"['1597190', '14666255', '14656365', '14979077'...",NaN,N/A,
3646,9011-04-5,https://drugs.ncats.io/drug/4C905MSK4W,"['HEXADIMETHRINE BROMIDE [INN]', ""POLY(N,N,N',...",HEXADIMETHRINE BROMIDE,['POLYBRENE'],719.0,24769,NaN,NaN,NaN,...,1959.0,[],[],[],[],[],[],NaN,N/A,
3647,8006-90-4,https://drugs.ncats.io/drug/AV092KU4JH,"['PEPPERMINT OIL', 'PEPPERMINT OIL YAKIMA', 'P...",PEPPERMINT OIL,[],NaN,6850741,NaN,NaN,NaN,...,1921.0,[],[],[],[],[],[],C015424,8006-90-4,
